In [74]:
import tifffile
from pathlib import Path
from cellpose import models
import numpy as np
from skimage import measure, exposure, filters
import pandas as pd
import napari


In [75]:
image = "data/IHC_1/AD1867/CA1/Image1.lsm"

# Extract filename, region, mouse and IHC round
file_path = Path(image)
file_path_parts = file_path.parts
filename = file_path.stem
region = file_path_parts[-2]
mouse_id = file_path_parts[-3]
ihc_round = file_path_parts[-4]

# Read the image file
img = tifffile.imread(image)

# Extract per channel info
nuclei_img = img[0,:,:]
h2a_img = img[1,:,:]
cfos_img = img[2,:,:]


In [76]:
cellpose_nuclei_diameter = 40
gaussian_sigma = 2

model = models.Cellpose(gpu=True, model_type="nuclei")

# Create a copy of nuclei_mip
input_img = nuclei_img.copy()

# Might need to perform a Gaussian-blur before
post_gaussian_img = filters.gaussian(
    input_img, sigma=gaussian_sigma
)

# Apply Contrast Stretching to improve Cellpose detection of overly bright nuclei
p2, p98 = np.percentile(post_gaussian_img, (2, 98))
img_rescale = exposure.rescale_intensity(
    post_gaussian_img, in_range=(p2, p98)
)

# Predict nuclei nuclei_masks using cellpose
nuclei_masks, flows, styles, diams = model.eval(
    img_rescale,
    diameter=cellpose_nuclei_diameter,
    channels=[0, 0],
    net_avg=False,
            )

In [77]:
# Extract regionprops from nuclei labels and H2A channel
h2a_props = measure.regionprops_table(
    label_image=nuclei_masks,
    intensity_image=h2a_img,
    properties=[
        "label",
        "intensity_mean",
        "intensity_max",
        "area_filled",
        "perimeter",
        "equivalent_diameter"
    ],
)

# Construct a dataframe
h2a_df = pd.DataFrame(h2a_props)

# Rename the intensity columns for further merging with cfos_df
h2a_df.rename(columns={'intensity_mean': 'h2a_intensity_mean'}, inplace=True)
h2a_df.rename(columns={'intensity_max': 'h2a_intensity_max'}, inplace=True)


In [78]:
# Extract regionprops from nuclei labels and CFOS channel
cfos_props = measure.regionprops_table(
    label_image=nuclei_masks,
    intensity_image=cfos_img,
    properties=[
        "label",
        "intensity_mean",
        "intensity_max"
    ],
)

# Construct a dataframe
cfos_df = pd.DataFrame(cfos_props)

# Rename the intensity columns for further merging with cfos_df
cfos_df.rename(columns={'intensity_mean': 'cfos_intensity_mean'}, inplace=True)
cfos_df.rename(columns={'intensity_max': 'cfos_intensity_max'}, inplace=True)

In [79]:
merged_df = pd.merge(cfos_df, h2a_df, on='label', how='inner')

In [80]:
# Create a new DataFrame with the same index as merged_df and the new columns
new_columns_df = pd.DataFrame({
    'filename': [filename] * len(merged_df),
    'region': [region] * len(merged_df),
    'mouse_id': [mouse_id] * len(merged_df),
    'ihc_round': [ihc_round] * len(merged_df)
}, index=merged_df.index)

# Concatenate the new columns DataFrame with the original merged_df
# Using pd.concat and specifying axis=1 for columns, and placing the new DataFrame first
merged_df = pd.concat([new_columns_df, merged_df], axis=1)


In [81]:
merged_df

,filename,region,mouse_id,ihc_round,label,cfos_intensity_mean,cfos_intensity_max,h2a_intensity_mean,h2a_intensity_max,area_filled,perimeter,equivalent_diameter
0,Image1,CA1,AD1867,IHC_1,1,13.225974,39.0,13.851299,58.0,1540.0,154.568542,44.280796
1,Image1,CA1,AD1867,IHC_1,2,18.909594,38.0,21.547355,83.0,1626.0,157.254834,45.500412
2,Image1,CA1,AD1867,IHC_1,3,19.713966,60.0,9.843778,62.0,2234.0,180.367532,53.333077
3,Image1,CA1,AD1867,IHC_1,4,16.776413,40.0,13.968468,77.0,2442.0,188.066017,55.760658
4,Image1,CA1,AD1867,IHC_1,5,18.487179,42.0,12.203360,59.0,1131.0,127.053824,37.947779
5,Image1,CA1,AD1867,IHC_1,6,18.338400,86.0,11.113645,87.0,2763.0,199.923882,59.312401
6,Image1,CA1,AD1867,IHC_1,7,14.571207,37.0,3.195753,33.0,1931.0,166.994949,49.584529
7,Image1,CA1,AD1867,IHC_1,8,7.513514,22.0,1.383784,11.0,185.0,48.627417,15.347616
8,Image1,CA1,AD1867,IHC_1,9,14.659259,42.0,14.409877,73.0,810.0,109.639610,32.114234
9,Image1,CA1,AD1867,IHC_1,10,17.158903,46.0,10.735554,53.0,3392.0,218.994949,65.717795


In [82]:
# Initialize napari.Viewer and display input stacks and label processing steps
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)
viewer.add_image(nuclei_img)
viewer.add_image(post_gaussian_img)
viewer.add_image(img_rescale)
viewer.add_labels(nuclei_masks)

<Labels layer 'nuclei_masks' at 0x2a42d12ed90>